In [1]:
from conda_build import api
import os
from pprint import pprint
import importlib
importlib.reload(api)

<module 'conda_build.api' from '/Users/msarahan/code/conda-build/conda_build/api.py'>

In [2]:
def print_yamls(recipe, **kwargs):
    yamls = [api.output_yaml(m[0]) 
             for m in api.render(recipe, verbose=False, permit_unsatisfiable_variants=True, **kwargs)]
    for yaml in yamls:
        print(yaml)

In [3]:
def print_outputs(recipe, **kwargs):
    pprint(api.get_output_file_path(recipe, verbose=False, **kwargs))

In [4]:
print_outputs('01_basic_templating/')

]0;IPython: code/demofriday20170203

MultipleInstanceError: Multiple incompatible subclass instances of TerminalIPythonApp are being created.

In [ ]:
print_yamls('01_basic_templating/')

In [ ]:
print_yamls('02_python_version/')

In [ ]:
# Setting environment variables overrides the conda_build_config.yaml
os.environ["CONDA_PY"] = "3.4"
print_yamls('02_python_version/')
del os.environ['CONDA_PY']

In [ ]:
# passing python as an argument also overrides conda_build_config.yaml
print_yamls('02_python_version/', python="3.6")

In [ ]:
outputs = api.build('02_python_version/', python="3.6", 
                    no_anaconda_upload=True)
outputs

In [ ]:
!conda inspect hash-inputs ~/miniconda3/conda-bld/osx-64/abc-1.0-py36h7d013e7_0.tar.bz2

In [ ]:
pprint(api.get_output_file_path('03_numpy_matrix/', verbose=False))

In [ ]:
print_yamls('03_numpy_matrix/')

In [ ]:
print_yamls('04_numpy_matrix_pinned/')

In [ ]:
print_yamls('05_compatible/')

In [ ]:
print_yamls('06_compatible_custom/')

In [ ]:
print_yamls('07_compatible_custom_lower_upper/')

In [ ]:
print_yamls('08_compiler/')

In [ ]:
print_yamls('09_cross/')

In [ ]:
# imposing runtime requirements on downstream packages
api.build('10_runtimes/pins_downstream')

In [ ]:
print_yamls('10_runtimes/uses_pinned_package')